In [59]:
import mysql.connector

connection = mysql.connector.connect(
  host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
  port = 4000,
  user = "2B3nARpwz82J49u.root",
  password = "tpc7Iw6jTcsoavz2",
  database = "TextToVector",
  ssl_ca = "isrgrootx1 (2).pem",
  ssl_verify_cert = True,
  ssl_verify_identity = True
)

cur = connection.cursor()

In [2]:
print(connection)

In [3]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Jupyter-specific imports
from IPython.display import display, Markdown

# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

c:\Users\User\anaconda3\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
local_path = 'scammer-agent.pdf'
if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path)
    data = loader.load()
    print(f'PDF loaded successfully: {local_path}')
else:
    print('Upload a PDF file')

PDF loaded successfully: scammer-agent.pdf


In [5]:
data

[Document(metadata={'source': 'scammer-agent.pdf'}, page_content='4 2 0 2\n\nt c O 1 2\n\n] I\n\nA . s c [\n\n1 v 0 5 6 5 1 . 0 1 4 2 : v i X r a\n\nVoice-Enabled AI Agents can Perform Common Scams\n\nRichard Fang UIUC\n\nDylan Bowman UIUC\n\nDaniel Kang UIUC\n\nAbstract\n\nRecent advances in multi-modal, highly ca- pable LLMs have enabled voice-enabled AI agents. These agents are enabling new applica- tions, such as voice-enabled autonomous cus- tomer service. However, with all AI capabili- ties, these new capabilities have the potential for dual use.\n\nIn this work, we show that voice-enabled AI agents can perform the actions necessary to perform common scams. To do so, we select a list of common scams collected by the govern- ment and construct voice-enabled agents with directions to perform these scams. We con- duct experiments on our voice-enabled agents and show that they can indeed perform the ac- tions necessary to autonomously perform such scams. Our results raise questions a

## Clean Text

In [47]:
import re
import unicodedata
def clean_text(data):
    # ลบเลขเดี่ยว ๆ และเครื่องหมายจุดสั้น ๆ
    text = re.sub(r'^\s*\d+\s*$', '', data, flags=re.MULTILINE)
    text = re.sub(r'^[^\w\s]{1,3}$', '', text, flags=re.MULTILINE)

    # ลบ arXiv/viXra
    text = re.sub(r'\b(arXiv|viXra):\s*\d+\.\d+\b', '', text, flags=re.IGNORECASE)

    # ลบหัวข้อ/section แบบ "2.", "3.1.2"
    text = re.sub(r'^\s*\d+(\.\d+)*\s*$', '', text, flags=re.MULTILINE)

    # ลบปีซ้ำ เช่น "2021. 2021."
    text = re.sub(r'\b(\d{4})\.\s*\1\.', r'\1.', text)

    # ลบหัวข้อ section เช่น "2 Common Scams ..."
    text = re.sub(r'^\d+(\.\d+)*\s+[A-Z][^\n]+', '', text, flags=re.MULTILINE)

    # ลบ citation "(Kang et al., 2024; ...)"
    text = re.sub(r'\([^)]*\d{4}[a-z]?\)', '', text)

    # ลบ bracket citations [1], [2, 3]
    text = re.sub(r'\[\d+(,\s*\d+)*\]', '', text)

    # ลบคำซ้ำต่อท้ายกัน เช่น "arXiv preprint . arXiv preprint ."
    text = re.sub(r'\b(\w+(?:\s+\w+){0,3})\.\s+\1\.', r'\1.', text)

    # ลบประโยคซ้ำทั้งประโยค
    text = re.sub(r'(\b.+?[.?!])(\s+\1)+', r'\1', text)

    # จัดการ newline + space
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)
    text = re.sub(r'\n{2,}', '\n\n', text)

    # ลบขีดคั่น
    text = re.sub(r'-\s+', '', text)

    # ลบ space แปลก ๆ
    text = re.sub(r'(?<=\b\w)\s(?=\w\b)', '', text)

    # ลบ Figure/Table
    text = re.sub(r'Figure\s*\d+.*', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Table\s*\d+.*', '', text, flags=re.IGNORECASE)

    # Normalize unicode
    text = unicodedata.normalize("NFKC", text)

    # ลบ space ซ้ำ
    text = re.sub(r'\s{2,}', ' ', text)

    return text.strip()


In [48]:
data_clean = []
for doc in data:
    raw_text = doc.page_content
    text = clean_text(raw_text)
    data_clean.append(text)

In [51]:
print(data_clean)

['4202 tcO12 ] I A . sc [ 1v05651 . 0142 : viXra Voice-Enabled AI Agents can Perform Common Scams Richard Fang UIUC Dylan Bowman UIUC Daniel Kang UIUC Abstract Recent advances in multi-modal, highly capable LLMs have enabled voice-enabled AI agents. These agents are enabling new applications, such as voice-enabled autonomous customer service. However, with all AI capabilities, these new capabilities have the potential for dual use. In this work, we show that voice-enabled AI agents can perform the actions necessary to perform common scams. To do so, we select a list of common scams collected by the government and construct voice-enabled agents with directions to perform these scams. We conduct experiments on our voice-enabled agents and show that they can indeed perform the actions necessary to autonomously perform such scams. Our results raise questions around the widespread deployment of voice-enabled AI agents. Introduction AI capabilities have advanced rapidly in the past few years

## Chunking


In [9]:
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import nltk

embed_model = SentenceTransformer("BAAI/bge-m3")

tokenizer = AutoTokenizer.from_pretrained("google/embeddinggemma-300m")  
llm = AutoModelForCausalLM.from_pretrained("google/embeddinggemma-300m")




Some weights of Gemma3ForCausalLM were not initialized from the model checkpoint at google/embeddinggemma-300m and are newly initialized: ['lm_head.weight', 'model.embed_tokens.weight', 'model.layers.0.input_layernorm.weight', 'model.layers.0.mlp.down_proj.weight', 'model.layers.0.mlp.gate_proj.weight', 'model.layers.0.mlp.up_proj.weight', 'model.layers.0.post_attention_layernorm.weight', 'model.layers.0.post_feedforward_layernorm.weight', 'model.layers.0.pre_feedforward_layernorm.weight', 'model.layers.0.self_attn.k_norm.weight', 'model.layers.0.self_attn.k_proj.weight', 'model.layers.0.self_attn.o_proj.weight', 'model.layers.0.self_attn.q_norm.weight', 'model.layers.0.self_attn.q_proj.weight', 'model.layers.0.self_attn.v_proj.weight', 'model.layers.1.input_layernorm.weight', 'model.layers.1.mlp.down_proj.weight', 'model.layers.1.mlp.gate_proj.weight', 'model.layers.1.mlp.up_proj.weight', 'model.layers.1.post_attention_layernorm.weight', 'model.layers.1.post_feedforward_layernorm.weig

In [52]:
text = " ".join(data_clean)
sentences = nltk.sent_tokenize(text)

embeddings = embed_model.encode(sentences, convert_to_tensor=True)

chunks = []
used = set()
for i in range(len(sentences)):
    if i in used:
        continue
    sim_scores = util.cos_sim(embeddings[i], embeddings)[0]
    best_match = torch.topk(sim_scores, k=2).indices.tolist()[1]
    used.add(i)
    used.add(best_match)
    combined = sentences[i] + " " + sentences[best_match]
    chunks.append(combined)


In [57]:
print(f'{len(sentences)},{type(sentences)}')

210,<class 'list'>


In [58]:
print(f'{len(chunks)},{type(chunks)}')

149,<class 'list'>


In [66]:
chunks

['4202 tcO12 ] I A . .',
 'sc [ 1v05651 . .',
 '0142 : viXra Voice-Enabled AI Agents can Perform Common Scams Richard Fang UIUC Dylan Bowman UIUC Daniel Kang UIUC Abstract Recent advances in multi-modal, highly capable LLMs have enabled voice-enabled AI agents. As we have shown, voice-enabled LLM agents can perform the actions necessary to perform common phone scams.',
 'These agents are enabling new applications, such as voice-enabled autonomous customer service. Combined, these capabilities allow for beneficial applications, such as autonomous customer service .',
 'However, with all AI capabilities, these new capabilities have the potential for dual use. However, as with all AI capabilities, these capabilities have the potential for dual use .',
 'In this work, we show that voice-enabled AI agents can perform the actions necessary to perform common scams. In this work, we investigate the question: can voice-enabled AI agents perform the tasks needed to conduct common scams?',
 'To d

In [65]:
import json

def add_document(text):
    embedding = embed_model.encode(text).tolist()
    cur.execute('INSERT INTO DocumentsTable (content, embedding) VALUES (%s, %s)',(text, json.dumps(embedding)))
    connection.commit()

for chunk in chunks:
    add_document(chunk)

cur.close()
connection.close()



OperationalError: 2013 (HY000): Lost connection to MySQL server during query